In [1]:
from pynq.overlays.base import BaseOverlay
import cv2
import numpy as np
from pynq.lib.video import *
from time import sleep
base = BaseOverlay("base.bit")
#加入自己ip后重构的overlay
from pynq.lib import MicroblazeLibrary
pAudio = base.audio

In [2]:

# monitor configuration: 640*480 @ 60Hz
screen_w=320
screen_h=240
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()


pAudio.select_microphone()


# monitor (output) frame buffer size
frame_out_w = screen_w
frame_out_h = screen_h

# camera (input) configuration
frame_in_w = screen_w
frame_in_h = screen_h
sleep(10)
# initialize camera from OpenCV
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
print("Capture device is open: " + str(videoIn.isOpened()))
np_frame=0

# -*- coding:utf-8 -*-
import os
# 检测人脸
def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier('/home/xilinx/jupyter_notebooks/base/video/data/'
    'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)
    if (len(faces) == 0):
        return None, None
    (x, y, w, h) = faces[0]
    return gray[y:y + w, x:x + h], faces[0]
 
# 该函数将读取所有的训练图像，从每个图像检测人脸并将返回两个相同大小的列表，分别为脸部信息和标签
def prepare_training_data(data_folder_path):
    dirs = os.listdir(data_folder_path)
    faces = []
    labels = []
    for dir_name in dirs:
        if not dir_name == '.ipynb_checkpoints':
            label = int(dir_name)
        subject_dir_path = data_folder_path + "/" + dir_name
        subject_images_names = os.listdir(subject_dir_path)
        for image_name in subject_images_names:
            if not dir_name == '.ipynb_checkpoints':
                image_path = subject_dir_path + "/" + image_name
                image = cv2.imread(image_path)
                face, rect = detect_face(image)
                if face is not None:
                    faces.append(face)
                    labels.append(label)
    #最终返回值为人脸和标签列表
    return faces, labels

#根据给定的（x，y）坐标和宽度高度在图像上绘制矩形
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x + w, y + h), (128, 128, 0), 2)

# 根据给定的（x，y）坐标标识出人名
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_COMPLEX, 1, (128, 128, 0), 2)

# 此函数识别传递的图像中的人物并在检测到的脸部周围绘制一个矩形及其名称
def predict(test_img):
    img = test_img.copy()
    face, rect = detect_face(img)
    if not face is None:
        label = face_recognizer.predict(face)
        if label[1]<140:
            label_text = subjects[label[0]]
        else:
            label_text = subjects[0]
        draw_rectangle(img, rect)
        draw_text(img, label_text, rect[0], rect[1] - 5)
    else:
        draw_text(img, 'nofound',0,20)

    return img

def imgtest(img):
    predicted_img = predict(img)


    return predicted_img

def imgSave(img,num):
    if not os.path.exists("./train/4"):
        os.mkdir("./train/4")
    face, rect = detect_face(img)
    if not face is None:
        cv2.imwrite("./train/4/"+str(num)+".jpg", img[:,:,[2,1,0]])
    return img, num

def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255), 1)

    
def get_distance():
    number=[0]
    distance_string=""#距离的字符串形式
    final_dis=0#距离的float类型
    isdata=0
    numcount=0
    lib = MicroblazeLibrary(base.RPI, ['uart'])
    device = lib.uart_open(14,15)
    D = [0x46]
    #控制命令
    lib.uart_write(device,D,len(D))
    sleep(0.1)
    while(1):
        lib.uart_read(device,number,len(number))
        if number[0]!=10 and number[0]!=58:
            if isdata and numcount!=0:
                distance_string=distance_string+chr(number[0])
                numcount = numcount-1
            if number[0]==13:
                isdata=1
                numcount=7
            if numcount==1:
                isdata=0
                distance_string=distance_string[0:6]
                distance_string = distance_string.replace(' ','')
                distance_string = distance_string.replace(',','')
                distance_string = distance_string.replace('m','')
                if ord(distance_string[0])>=48 and ord(distance_string[0])<58:
                    final_dis=float(distance_string)
                    device.close()
                    return final_dis
                else:
                    return 0

            
def get_voltage():
    #从自己加入的ip获取电压值
    FS=4.096
    while 1:  
        input=base.rgbleds_gpio.channel2.read()
        if input>0x8000:
            input = 0x10000 - input
            ADS1118_Voltage=-1*(input*FS / 0x8000)
        else:
            ADS1118_Voltage = input*FS /0x8000
        return ADS1118_Voltage

Capture device is open: True


In [ ]:
  
####################################################################################################       
stage = 0
num = 1
face_recognizer = cv2.face.createLBPHFaceRecognizer()
#建立标签与人名的映射列表（标签只能为整数）
subjects = ["Unknown","Zhong", "Pan","Huang","testperson"]
faces, labels = prepare_training_data("train")
face_recognizer.train(faces, np.array(labels))
nose_cascade = cv2.CascadeClassifier('/home/xilinx/jupyter_notebooks/base/video/data/''haarcascade_mcs_nose.xml')
outframe = hdmi_out.newframe()
distance = 0
voltage = 0
temperature = 0
while True:
    if base.buttons[0].read()==1 and base.switches.read()==0:
        stage=1#口罩识别
    if base.buttons[1].read()==1 and base.switches.read()==0:
        stage=2#人脸检测
    if base.buttons[2].read()==1 and base.switches.read()==0:
        stage=3#人脸学习
    if base.buttons[3].read()==1 and base.switches.read()==0:
        stage=0#直接显示
    if base.switches.read()==1:
        stage=4#温度显示
        
    if stage == 0:
        ret, np_frame = videoIn.read()
        outframe[0:screen_h,0:screen_w,:] = np_frame[0:screen_h,0:screen_w,:]
        hdmi_out.writeframe(outframe)
    if stage == 1:
        ret, np_frame = videoIn.read()
        gray = cv2.cvtColor(np_frame, cv2.COLOR_BGR2GRAY)
        noses = nose_cascade.detectMultiScale(gray)
        if(len(noses)>=1):
            draw_text(np_frame, 'Nomask',0,20)
        outframe[0:screen_h,0:screen_w,:] = np_frame[0:screen_h,0:screen_w,:]
        hdmi_out.writeframe(outframe)
    if stage == 2:
        ret, np_frame = videoIn.read()
        np_frame = imgtest(np_frame)
        outframe[0:screen_h,0:screen_w,:] = np_frame[0:screen_h,0:screen_w,:]
        hdmi_out.writeframe(outframe)
    if stage == 3:
        ret, np_frame = videoIn.read()
        outframe = hdmi_out.newframe()
        outframe, num = imgSave(outframe, num)
        draw_text(outframe, 'num:'+str(num),0,20)
        print(num)
        num = num + 1
        outframe[0:screen_h,0:screen_w,:] = np_frame[0:screen_h,0:screen_w,:]
        hdmi_out.writeframe(outframe)
        if num>10:
            faces, labels = prepare_training_data("train")
            face_recognizer.train(faces, np.array(labels))
            stage = 0
            num = 1
    if stage ==4:
        ret, np_frame = videoIn.read()
        distance = get_distance()
        voltage = get_voltage()
        temperature =  30.47- 3.451*distance+3.093*voltage
        draw_text(np_frame, 'distance:'+str(distance)+'m',0,20)
        #draw_text(np_frame, 'voltage:'+str(voltage)[0:6]+'V',0,40)
        draw_text(np_frame, 'temperature:'+str(temperature)[0:6]+'C',0,40)
        if temperature>37:
            pAudio.load("/home/xilinx/jupyter_notebooks/base/audio/temp.wav")
            pAudio.play()
        outframe[0:screen_h,0:screen_w,:] = np_frame[0:screen_h,0:screen_w,:]
        hdmi_out.writeframe(outframe)

In [42]:
videoIn.release()
hdmi_out.stop()
del hdmi_out
device.close()

NameError: name 'device' is not defined

In [5]:
lib = MicroblazeLibrary(base.RPI, ['uart'])
device = lib.uart_open(14,15)
device.close()